In [2]:
import os
import sys
import re
from pathlib import Path

from IPython.display import display, HTML, Markdown
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Project level imports
from larval_gonad.notebook import Nb

In [3]:
# Setup notebook
nbconfig = Nb.setup_notebook(seurat_dir='../scrnaseq-wf/data/scrnaseq_combine_force')

last updated: 2018-08-23 
Git hash: d860e817a227b978bd35bac75b3a4e77c27a36a0


In [21]:
chroms = nbconfig.fbgn2chrom
chroms.chrom = chroms.chrom.apply(lambda x: x.lstrip('chr'))

In [4]:
raw = pd.read_parquet('../scrnaseq-wf/data/raw_by_cluster.parquet')

In [7]:
germ = raw.iloc[:, :5]

In [28]:
germ_melted = germ.reset_index().melt(id_vars='FBgn', var_name='cluster', value_name='raw').set_index('FBgn')
germ_melted = germ_melted.join(chroms)

In [35]:
# add x-A mapptings
x = germ_melted.chrom == 'X'
a = germ_melted.chrom.isin(['2L', '2R', '3L', '3R'])

germ_melted.loc[x, 'XA'] = 'X'
germ_melted.loc[a, 'XA'] = 'A'

In [46]:
germ_melted.XA.value_counts()

A    60540
X    10945
Name: XA, dtype: int64

In [47]:
germ_melted.chrom.value_counts()

3R    17550
3L    14515
2R    14485
2L    13990
X     10945
4       510
Y       225
M        90
Name: chrom, dtype: int64

In [54]:
unique = germ[(germ == 0).sum(axis=1) == 4]

In [62]:
unique = unique.reindex(unique.max(axis=1).sort_values(ascending=False).index)

In [65]:
unique = unique.assign(gene_symbol=[nbconfig.fbgn2symbol[x] for x in unique.index]).set_index('gene_symbol', append=True)

In [66]:
unique.to_csv('../output/germcell_unique_genes.tsv', sep='\t')